In [ ]:
# check against table 7

In [1]:
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures


import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load the base mechanism

# Load the model
cti_path = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model/chem_annotated.cti'
# cti_path = '/work/westgroup/harris.se/autoscience/aramco/AramcoMech3.0.MECH.cti'
cti_path = '/scratch/harris.se/autoscience/naive_model/changed_models/chem_2022-08-29.cti'
cti_path = '/work/westgroup/harris.se/autoscience/autoscience/butane/improved_models/chem_2022-09-08.cti'


gas = ct.Solution(cti_path)

$C_4H_{10} + \frac{13}{2}O_2 \leftrightharpoons 4CO_2 + 5H_2O\$

In [4]:
# load the experimental conditions
flame_speed_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/butane_flamespeeds.csv'
df_exp = pd.read_csv(flame_speed_data)

# get just the Park data
data_slice = df_exp[df_exp['Reference'] == 'Park et al. 2016']

# Define Initial conditions using experimental data
speeds = data_slice['SL0 (cm/s)'].values.astype(float)  # ignition delay
temperatures = data_slice['Tu (K)'].values  # Temperatures
pressures = data_slice['Pu (atm)'].values * ct.one_atm  # pressures in atm
equiv_ratios = data_slice['Equivalence Ratio'].values  # equivalence ratio


# list of starting conditions
# Define stoichiometric coefficients
v_fuel = 1.0
v_oxidizer = 13.0 / 2.0
v_N2 = 0.79 * (v_oxidizer / 0.21)  # air is approximately 79% N2 and 21% O2

# calculate actual ratio of fuel to oxidizer
actual_ratio = equiv_ratios * (v_fuel / v_oxidizer)


# start with 1.0 oxidizer, then normalize
x_O2 = 1.0
x_C4H10 = actual_ratio * x_O2
x_N2 = 0.79 * (x_O2 / .21)
total = x_O2 + x_C4H10 + x_N2
x_O2 = x_O2 / total
x_C4H10 = x_C4H10 / total
x_N2 = x_N2 / total

concentrations = [{'butane(1)': x_C4H10[i], 'O2(2)': x_O2[i], 'N2': x_N2[i]} for i in range(0, len(equiv_ratios))]

In [5]:
# function for running a flame speed
# assumes gas has been properly initialized
def run_flame_speed(condition_index):
    
    gas = ct.Solution(cti_path)
    
    gas.TPX = temperatures[condition_index], pressures[condition_index], concentrations[condition_index]

    
    tol_ss = [1.0e-13, 1.0e-9]  # abs and rel tolerances for steady state problem
    tol_ts = [1.0e-13, 1.0e-9]  # abs and rel tie tolerances for time step function

    width = 0.08
    flame = ct.FreeFlame(gas, width=width)
    flame.flame.set_steady_tolerances(default=tol_ss)   # set tolerances
    flame.flame.set_transient_tolerances(default=tol_ts)
    flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
    flame.max_time_step_count = 900
    loglevel = 1

    print("about to solve")
    flame.solve(loglevel=loglevel, auto=True)
    Su = flame.velocity[0]
    
    return Su

In [6]:
# # run the flame speeds
# for i in range(0, 1):
#     gas = ct.Solution(cti_path)
    
#     gas.TPX = temperatures[i], pressures[i], concentrations[i]
    
#     Su = run_flame_speed(gas)
#     print()

In [8]:
len(data_slice)

12

In [ ]:
# Run all simulations in parallel
flame_speeds = np.zeros(len(data_slice))
# condition_indices = np.arange(0, len(data_slice))
condition_indices = np.arange(0, len(data_slice))
with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
    for condition_index, flame_speed in zip(condition_indices, executor.map(run_flame_speed, condition_indices)):
        flame_speeds[condition_index] = flame_speed

about to solveabout to solveabout to solveabout to solveabout to solveabout to solveabout to solveabout to solveabout to solveabout to solveabout to solve


about to solve









************ Solving on 8 point grid with energy equation enabled ************
************ Solving on 8 point grid with energy equation enabled ************
************ Solving on 8 point grid with energy equation enabled ************
************ Solving on 8 point grid with energy equation enabled ************


************ Solving on 8 point grid with energy equation enabled ************



************ Solving on 8 point grid with energy equation enabled ************


************ Solving on 8 point grid with energy equation enabled ************

************ Solving on 8 point grid with energy equation enabled ************
..............................................................................
..............................................................................
************ Solving 

..............................................................................
    failure. 
grid refinement disabled.
Take 10 timesteps   
******************** Solving with grid refinement enabled ********************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 1 2 3 4 5 
    to resolve C2H2(25) C2H3(22) C2H4(11) C2H4O(51) C2H5(33) C2H5O2(47) C2H6(32) C2H6O2(48) C3H4O(74) C3H5-A(94) C3H6(12) C4H6(194)   1.144e-08      5.207
C4H7(190) Attempt Newton solution of steady-state problem...C4H8(188) C4H8(189) C4H8O(214) CH2(23) CH2CO(24) CH2O(9) CH3(18) CH3CHO(35) CH3O(31) CH3O2(45) CH3O2H(46) CH3OH(30) CH4(10) CO(6) CO2(7) H(14) H2(13)

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     8.109e-05      5.487
Attempt Newton solution of steady-state problem...  3.624e-08      4.076
Attempt Newton solution of steady-state problem...  2.281e-05      6.095
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     1.507e-07      4.756
Attempt Newton solution of steady-state problem...  9.289e-09      6.504
Attempt Newton solution of steady-state problem...  1.881e-07      5.442
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     6.703e-09      5.331
Attempt Newton solution of steady-state problem...  3.711e-05      5.763
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     2.718e-08      5.211
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     3.865e-07      5.33

Take 10 timesteps     1.224e-08      6.519
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      1.55e-08      5.437
Attempt Newton solution of steady-state problem...  3.976e-08      3.797
Attempt Newton solution of steady-state problem...  7.961e-09      4.447
    failure. 
Attempt Newton solution of steady-state problem...Take 10 timesteps       failure. 
Take 10 timesteps     4.478e-09      5.214
Attempt Newton solution of steady-state problem...  8.936e-07      5.434
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     2.152e-08       4.07
Attempt Newton solution of steady-state problem...  5.153e-05      5.264
  6.369e-08       5.44
Attempt Newton solution of steady-state problem...Attempt Newton solution of steady-state problem...  1.789e-07      4.757
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     2.582e-07 

Attempt Newton solution of steady-state problem...    success.

Problem solved on [20] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 5 6 7 8 9 10 
    to resolve C2H(4) C2H2(25) C2H3(22) C2H3O3(56) C2H4(11) C2H4O(51) C2H5(33) C2H5O2(47) C2H6(32) C2H6O2(48) C3H4O(74) C3H5-A(94) C3H6(12) C3H7O2(95) C3H7O2(97) C4H6(194) C4H6(2530) C4H7(190) C4H7(191) C4H7O(202) C4H8(188) C4H8(189) C4H8O(214) CH2(23) CH2CHO(21) CH2CO(24) CH2O(9) CH3(18) CH3CHO(35) CH3CO(20) CH3O(31) CH3O2(45) CH3O2H(46) CH3OH(30) CH4(10) CO(6) CO2(7) H(14) H2(13) H2O(8) H2O2(17) HCO(19) HO2(16) HOCHO(40) IC3H7(93) N2 NC3H7(92) O(5) O2(2) OH(15) PC4H9(182) S(184) S(186) S(187) S(256) S(3513) S(4243) S(4244) S(4407) S(5151)     failure. 
S(5425) Take 10 timesteps   S(5761) S(787) S(9510) SC4H9(183) T butane(1) velocity 
##############

...

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     6.277e-08      5.222
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     7.463e-08      1.569
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.893e-05      6.201
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     8.833e-05      5.621
Attempt Newton solution of steady-state problem...   6.53e-08      5.979
Attempt Newton solution of steady-state problem...  5.656e-08      5.222
Attempt Newton solution of steady-state problem...  5.084e-05      5.967
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     6.724e-08      1.569
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     9.125e-08      5.446
Att

    New points inserted after grid points 1 2 3 4 5 
    to resolve C2H(4) C2H2(25) C2H3(22) C2H4(11) C2H4O(51) C2H5(33) C2H5O2(47) C2H6(32) C2H6O2(48) C3H4O(74) C3H5-A(94) C3H6(12) C4H6(194) C4H7(190) C4H8(188) C4H8(189) C4H8O(214) CH2(23) CH2CO(24) CH2O(9) CH3(18) CH3CHO(35) CH3O(31) CH3O2(45) CH3O2H(46) CH3OH(30) CH4(10) CO(6) CO2(7) H(14) H2(13) H2O(8) H2O2(17) HCO(19) HO2(16) HOCHO(40) N2 O(5) O2(2) OH(15) S(184) S(186) S(187) S(3513) S(4243) S(4244) S(4407) S(5151) S(5761) S(787) SC4H9(183) T butane(1) velocity 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...  1.544e-05       6.67
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     8.344e-09      5.445
Attempt Newton solution of steady-state problem...  8.798e-07      4.181
Attempt Newton solution of steady-state problem...    failure. 


    to resolve C2H(4) C2H2(25) C2H3(22) C2H3O3(56) C2H4(11) C2H4O(51) C2H5(33) C2H5O(49) C2H5O2(47) C2H6(32) C2H6O2(48) C3H4O(74) C3H5-A(94) C3H5O(131) C3H6(12) C3H7O2(95) C3H7O2(97) C4H6(194) C4H6(2530) C4H6(2534) C4H7(190) C4H7(191) C4H7(192) C4H7(4343) C4H7O(202) C4H8(188) C4H8(189) C4H8O(214) CH(3) CH2(23) CH2CHO(21) CH2CO(24) CH2O(9) CH3(18) CH3CHO(35) CH3CO(20) CH3O(31) CH3O2(45) CH3O2H(46) CH3OH(30) CH4(10) CO(6) CO2(7) H(14) H2(13) H2O(8) H2O2(17) HCO(19) HO2(16) HOCHO(40) IC3H7(93) N2 NC3H7(92) O(5) O2(2) OH(15) PC4H9(182) S(184) S(186) S(187) S(219) S(223) S(256) S(3513) S(4243) S(4244) S(4407) S(5151) S(5425) S(5761) S(787) S(9510) SC4H9(183) T butane(1) velocity 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     4.777e-08      4.176
Attempt Newton solution of steady-state problem...

    failure. 
Take 10 timesteps     6.606e-09      5.443
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      4.426
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps     9.691e-05      5.638
Attempt Newton solution of steady-state problem...   0.001095      3.988
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       failure. 
Take 10 timesteps       success.

Problem solved on [26] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 8 9 10 11 12 13 14 
    to resolve C2H(4) C2H2(25) C2H3(22) C2H3O3(56) C2H4(11) C2H4O(51) C2H5(33) C2H5O2(47) C2H6(32) C2H6O2(48) C3H4O(74) C3H5-A(94) C3H6(12) C3H7O2(95) C3H7O2(97) C4H6(194) C4H6(2530) C4H7(190) C4H7(191) C4H8

##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      3.01e-07      3.787
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.879e-08      4.408
Attempt Newton solution of steady-state problem...    success.

Problem solved on [49] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 17 18 19 20 21 22 23 24 25 26 27 28 
    to resolve C2H(4) C2H2(25) C2H3(22) C2H3O3(56) C2H4(11) C2H4O(51) C2H5(33) C2H5O2(47) C2H6(32) C2H6O2(48) C3H4O(74) C3H5-A(94) C3H6(12) C3H7O2(95) C3H7O2(97) C4H6(194) C4H6(2530) C4H7(190) C4H7(191) C4H7(192) C4H7(4343) C4H8(188) C4H8(189) C4H8O(214) CH2(23) 

In [15]:
out_df = pd.DataFrame(speeds)
out_df.to_csv('flame_speeds_20220908.csv')

In [ ]:
delays

In [ ]:
# Saved ignition delays for comparison:
base_rmg7 = np.array([0.04908328, 0.04854439, 0.04814446, 0.04868176, 0.075588  ,
       0.07737167, 0.14791668, 0.14857535, 0.1085917 , 0.0952434 ,
       0.06520823, 0.05563237, 0.03535775, 0.02462961, 0.02334903,
       0.01398333])  # 24 hour RMG run - 110 species

full_base_rmg7 = np.array([0.04830086, 0.04773837, 0.0462931 , 0.04678523, 0.07270171,
       0.07446053, 0.14525752, 0.14593983, 0.10720529, 0.09406754,
       0.0644564 , 0.05500488, 0.03497752, 0.02436985, 0.02310303,
       0.01383285])  # 1 week RMG model that completed - all 130 species

aramco7 = np.array([0.03945921, 0.03874611, 0.02817182, 0.02812066, 0.03236667,
       0.03278732, 0.06009581, 0.06085926, 0.06020536, 0.05459522,
       0.03965133, 0.03435827, 0.02245863, 0.01580939, 0.01499945,
       0.00896072])

In [ ]:
# plot the 
plt.plot(1000.0 / T7, delays, marker='x')
plt.plot(1000.0 / T7, base_rmg7, marker='x', linestyle='dashed')
plt.plot(1000.0 / T7, aramco7, marker='x')
plt.scatter(1000.0 / T7, tau7 / 1000.0, color='black')
ax = plt.gca()
ax.set_yscale('log')
# plt.legend(['Mechanism 174', 'Base RMG', 'Aramco', 'Experiment'])
plt.legend(['this', 'Base RMG', 'Aramco', 'Experiment'])
plt.title('Ignition Delays Phi=1.0')
plt.xlabel('1000K / T')
plt.ylabel('Delay (s)')
plt.savefig('table7.png')

In [ ]:
delays

In [ ]:
delays[7]